# Visualization of Land Info

Using Ministry of Land, Infrastructure, Transport and Tourism, Japan.

Special Thanks
* http://qiita.com/shima_x/items/fe29274d67de3a461524
* http://sinhrks.hatenablog.com/entry/2015/06/14/215514

## logger

In [1]:
from logging import getLogger, StreamHandler, INFO, NullHandler
logger = getLogger(__name__)
# handler = StreamHandler()
handler = NullHandler()
handler.setLevel(INFO)
logger.setLevel(INFO)
logger.addHandler(handler)

logger.debug('hello')

In [2]:
import os
import osgeo.ogr
import shapefile
import json
from json import dumps
from IPython.display import HTML
import folium
import pandas as pd
import numpy as np

oita_webland_data_dir = "./webland_oita/"
input_shape_file = "./N03-20150101_44_GML/N03-15_44_150101.shp"
intermediate_geojson_file = "./oita-geo.json"

## shapefile to geojson

Convert a shapefile to a dict variable for the purpose of making a geojson file.

In [3]:
shape_reader = shapefile.Reader(input_shape_file)

# extract field names
fields = shape_reader.fields[1:]
field_names = [field[0] for field in fields]

out_buffer = []
for sr in shape_reader.shapeRecords():
    atr = dict(zip(field_names, sr.record))
    geom = sr.shape.__geo_interface__
    out_buffer.append(dict(type="Feature",
                       geometry=geom, properties=atr))

Convert multi-byte strings to those of utf-8, because multi-byte strings are recognized as bytes string type.

In [4]:
for out_buffer_item in out_buffer:
    for prop_key, prop_val in out_buffer_item["properties"].items():
        logger.debug("key: {}, value: {}".format(prop_key, prop_val))        
        if type(prop_val) == bytes:
            logger.debug("str: {}".format(prop_val.decode('shift-jis')))
            out_buffer_item["properties"][prop_key] = prop_val.decode('shift-jis')

output to json

In [5]:
geojson_str = dumps({"type": "FeatureCollection",
                     "features": out_buffer},
                    sort_keys=True, ensure_ascii=False, indent=2)

with open(intermediate_geojson_file, "w") as fh:
    logger.debug("output type: {}".format(type(geojson_str)))
    fh.write(geojson_str)

## folium
### Trial Plot

inline visualization function for ipython notebook

In [6]:
# folium.initialize_notebook()  # is not available at 2015/11/07

def inline_map(m, out_html_str='tmp.html'):
    # thanks to http://sinhrks.hatenablog.com/entry/2015/06/14/215514
    m.create_map(path=out_html_str)
    iframe = '<iframe srcdoc=\"{srcdoc}\" style=\"width: 100%; height: 400px; border: none\"></iframe>'
    return HTML(iframe.format(srcdoc=m.HTML.replace('\"', '&quot;')))

In [7]:
m = folium.Map(location=[33.2382026, 131.612535], zoom_start=8)
m.simple_marker([33.2382026, 131.612535], popup='Oita')

inline_map(m, "trial_oita.html")

# if folium.initialize_notebook() is available, then following scripts will work

# m.create_map("trial_oita.html")
# m.render_iframe = True
# m

### input other (open) data via pandas 

importing csv files (Property price history, open data)

In [8]:
arr_df_webland_oita = []
# find csv files and append to array
for file in os.listdir(oita_webland_data_dir):
    if file.endswith(".csv"):
        df_tmp = pd.read_csv(oita_webland_data_dir+file,
                             encoding="cp932",
                             index_col=0)
        arr_df_webland_oita.append(df_tmp)

df_webland_oita = pd.concat(arr_df_webland_oita, ignore_index=True)

summarising by city_code (price per about 3.3 m^2)

In [9]:
df_oita_median_by_city = df_webland_oita.groupby("市区町村コード").median()
df_oita_median_by_city["city_code"] = df_oita_median_by_city.index

### using dataframe and geojson visualization

In [10]:
m = folium.Map(location=[33.2382026, 131.612535], zoom_start=8)
m.geo_json(geo_path=intermediate_geojson_file, data=df_oita_median_by_city,
    columns=['city_code', '取引価格（㎡単価）'],
    key_on='feature.properties.N03_007',
    threshold_scale=[3000, 9000, 15000, 24000, 30000],
    fill_color='BuPu', reset=True,
    legend_name='price_per_area')

inline_map(m)